<a href="https://colab.research.google.com/github/Suren1206/Titanic_RFC/blob/main/003_Titanic_Cleancopy_RFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
import pandas as pd

# Upload files
uploaded = files.upload()

# Load into pandas
train_df = pd.read_csv("train.csv")

train_df['Age'] = train_df.groupby(['Sex', 'Pclass'])['Age'].transform(
    lambda grp: grp.fillna(grp.median())
)
train_df['HasCabin'] = train_df['Cabin'].notnull().astype(int)
train_df = train_df.drop(columns=['Cabin'])
train_df['Embarked'] = train_df['Embarked'].fillna('S')

train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
train_df = pd.get_dummies(train_df, columns=['Embarked'], drop_first=False)
train_df = pd.get_dummies(train_df, columns=['Pclass'], drop_first=False)

# Create bins and labels for fare & create new column
fare_bins = [0, 25, 50, 100, train_df['Fare'].max()]
fare_labels = ['Low', 'Medium', 'High', 'Very High']
train_df['FareBand'] = pd.cut(train_df['Fare'], bins=fare_bins, labels=fare_labels)

# Create bins and labels for SibSp & create new column
sibsp_bins = [0, 1, 2, 4, train_df['SibSp'].max()]
sibsp_labels = ['Alone', 'Small', 'Medium', 'Large']
train_df['SibSpBand'] = pd.cut(train_df['SibSp'], bins=sibsp_bins, labels=sibsp_labels)

# Define bins and labels & create new column
parch_bins = [0, 1, 3, train_df['Parch'].max()]
parch_labels = ['Alone', 'Small', 'Large']
train_df['ParchBand'] = pd.cut(train_df['Parch'], bins=parch_bins, labels=parch_labels)

# Get one hot encoded values for all the 3 newly created features to make it more meaningful & drop the original features
train_df = pd.get_dummies(train_df, columns=['FareBand','SibSpBand','ParchBand'], drop_first=False)
train_df = train_df.drop(columns=['SibSp','Fare','Parch'])

# check for duplicates before model building
#  JUST FOR INFO ---- train_df.isnull().sum()

# Exclude these from the feature set
drop_cols = ['PassengerId', 'Name', 'Ticket', 'Survived']

# Select all other columns as features
features = train_df.drop(columns=drop_cols)

# Target variable
target = train_df['Survived']

# Step 1: Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Step 2: Prepare features and target
features = train_df.drop(columns=drop_cols)
target = train_df['Survived']

# Step 3: Train-test split
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.2, random_state=42)

# Step 4: Train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
rf.fit(X_train, y_train)

# Step 5: Evaluate on validation set
val_preds = rf.predict(X_val)
print("✅ Accuracy:", accuracy_score(y_val, val_preds))
print("\n✅ Confusion Matrix:\n", confusion_matrix(y_val, val_preds))
print("\n✅ Classification Report:\n", classification_report(y_val, val_preds))

# Optional Step 6: Predict on test set and prepare submission (if test_df is ready)
# test_features = test_df[features.columns]
# test_preds = rf.predict(test_features)
# submission = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': test_preds})
# submission.to_csv('submission.csv', index=False)

Saving train.csv to train (1).csv
✅ Accuracy: 0.8044692737430168

✅ Confusion Matrix:
 [[91 14]
 [21 53]]

✅ Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.87      0.84       105
           1       0.79      0.72      0.75        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.80       179
weighted avg       0.80      0.80      0.80       179

